In [ ]:
print("Hello World")

In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [ ]:
extracted_data=load_pdf_file(data='Data/')


In [ ]:
extracted_data

In [ ]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

In [ ]:
text_chunks

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [ ]:
query_result

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "chatbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [ ]:
retrieved_docs = retriever.invoke("What is the name of the chairman?")


In [ ]:
retrieved_docs

In [ ]:
import subprocess
import json

def query_ollama(prompt, context="", model="gemma3"):
    """Query the Ollama CLI with a prompt and context."""
    command = ["ollama", "run", model]
    input_data = {"messages": [
        {"role": "system", "content": context},
        {"role": "user", "content": prompt}
    ]}
    process = subprocess.run(command, input=json.dumps(input_data), text=True, capture_output=True)
    if process.returncode != 0:
        raise RuntimeError(f"Ollama CLI error: {process.stderr}")
    response = process.stdout
    return response

In [ ]:
def create_ollama_retrieval_chain(retriever, context_template):
    def retrieval_chain(input_query):
        # Retrieve relevant documents
        retrieved_docs = retriever.invoke(input_query)
        context = "\n".join([doc.page_content for doc in retrieved_docs])

        # Query Ollama with the context and input query
        response = query_ollama(input_query, context=context_template.format(context=context))
        return response

    return retrieval_chain

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n{context}"
)

rag_chain = create_ollama_retrieval_chain(retriever, context_template=system_prompt)
response = rag_chain("What is the fee structure ?")
print(response)